In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
from scipy.stats.mstats import winsorize
import scipy.stats
import statistics

warnings.filterwarnings('ignore')

In [4]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

usedu_df = pd.read_sql_query('select * from useducation',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [5]:
# using pandas interpolation function to fill missing values
fill_cols = usedu_df.columns[3:]
states = usedu_df['STATE'].unique()

# save empty enteries stats
before_inter = (usedu_df.isnull().sum()/usedu_df.shape[0]*100).values

for state in states:
    usedu_df.loc[usedu_df['STATE'] == state, fill_cols] = usedu_df.loc[usedu_df['STATE'] == state, fill_cols].interpolate()

# save empty entries stats after interpolation
after_inter = (usedu_df.isnull().sum()/usedu_df.shape[0]*100).values

# remaining missing values will be dropped
usedu_df.dropna(inplace=True)

# save empty entries stats after dropping na's
after_drop = (usedu_df.isnull().sum()/usedu_df.shape[0]*100).values

# print percent of empty cells before and after imputation
impute_ind = (usedu_df.isnull().sum()/usedu_df.shape[0]*100).index
print('{0:<30} {1:<12.3} {2:<12.3} {3:<12.3}'.format('', 'before', 'after interpolation', "after dropping na's")) 
for row in zip(impute_ind, before_inter, after_inter, after_drop):
    print('{0:<30} {1:<12.3} {2:<12.3} {3:<12.3}'.format(*row))

                               bef          aft          aft         
PRIMARY_KEY                    0.0          0.0          0.0         
STATE                          0.0          0.0          0.0         
YEAR                           0.0          0.0          0.0         
ENROLL                         17.6         14.1         0.0         
TOTAL_REVENUE                  14.2         10.8         0.0         
FEDERAL_REVENUE                14.2         10.8         0.0         
STATE_REVENUE                  14.2         10.8         0.0         
LOCAL_REVENUE                  14.2         10.8         0.0         
TOTAL_EXPENDITURE              14.2         10.8         0.0         
INSTRUCTION_EXPENDITURE        14.2         10.8         0.0         
SUPPORT_SERVICES_EXPENDITURE   14.2         10.8         0.0         
OTHER_EXPENDITURE              17.6         14.1         0.0         
CAPITAL_OUTLAY_EXPENDITURE     14.2         10.8         0.0         
GRADES_PK_G         

In [ ]:
# calculate math & reading weighted means
